# Pinterest User Engagement Analysis - Part 2: Engagement Metrics & User Segmentation
## Deep Dive into User Behavior and Content Performance

**Author:** Michael Oyetibo  
**Date:** January 2026  

---

### Objectives
1. Calculate and analyze engagement rate metrics
2. Identify high-performing content
3. Segment users based on behavior patterns
4. Provide actionable recommendations

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load datasets
users_df = pd.read_csv('../data/raw/users.csv')
pins_df = pd.read_csv('../data/raw/pins.csv')
interactions_df = pd.read_csv('../data/processed/interactions_enhanced.csv')
interactions_df['timestamp'] = pd.to_datetime(interactions_df['timestamp'])

print("Data loaded successfully!")
print(f"Users: {len(users_df):,}")
print(f"Pins: {len(pins_df):,}")
print(f"Interactions: {len(interactions_df):,}")

## 2. Calculate Pin-Level Engagement Metrics

In [ ]:
# Calculate engagement metrics for each pin
pin_metrics = interactions_df.groupby('pin_id').agg({
    'interaction_id': 'count',  # Total interactions
    'user_id': 'nunique'  # Unique users
}).rename(columns={'interaction_id': 'total_interactions', 'user_id': 'unique_users'})

# Count each interaction type
interaction_counts = interactions_df.groupby(['pin_id', 'interaction_type']).size().unstack(fill_value=0)
pin_metrics = pin_metrics.join(interaction_counts)

# Calculate engagement rate
# Engagement Rate = (saves + repins + clicks) / impressions
pin_metrics['engagement_actions'] = (
    pin_metrics.get('save', 0) + 
    pin_metrics.get('repin', 0) + 
    pin_metrics.get('click', 0)
)
pin_metrics['engagement_rate'] = (
    pin_metrics['engagement_actions'] / pin_metrics.get('impression', 1)
) * 100

# Replace infinity with 0
pin_metrics['engagement_rate'] = pin_metrics['engagement_rate'].replace([np.inf, -np.inf], 0)

# Join with pins data
pin_performance = pins_df.merge(pin_metrics, left_on='pin_id', right_index=True, how='left')
pin_performance = pin_performance.fillna(0)

print("✓ Pin-level engagement metrics calculated!")
print(f"\nEngagement Rate Statistics:")
print(pin_performance['engagement_rate'].describe())

## 3. Identify Top Performing Content

In [ ]:
# Top 20 pins by engagement rate
top_pins = pin_performance.nlargest(20, 'engagement_rate')[[
    'pin_id', 'category', 'content_type', 'has_description', 'has_link',
    'total_interactions', 'engagement_rate', 'impression', 'save', 'repin'
]]

print("Top 20 Pins by Engagement Rate:")
display(top_pins)

# Analyze characteristics of top performers
print("\nCharacteristics of Top Performers:")
print(f"Average Engagement Rate: {top_pins['engagement_rate'].mean():.2f}%")
print(f"\nCategory Distribution:")
print(top_pins['category'].value_counts())
print(f"\nContent Type Distribution:")
print(top_pins['content_type'].value_counts())
print(f"\nWith Description: {top_pins['has_description'].sum()}/{len(top_pins)}")
print(f"With Link: {top_pins['has_link'].sum()}/{len(top_pins)}")

## 4. Category Performance Analysis

In [ ]:
# Analyze performance by category
category_performance = pin_performance.groupby('category').agg({
    'pin_id': 'count',
    'total_interactions': 'sum',
    'engagement_rate': 'mean',
    'save': 'sum',
    'repin': 'sum',
    'click': 'sum'
}).rename(columns={'pin_id': 'total_pins'})

category_performance['avg_interactions_per_pin'] = (
    category_performance['total_interactions'] / category_performance['total_pins']
)

category_performance = category_performance.sort_values('engagement_rate', ascending=False)

print("Category Performance Analysis:")
display(category_performance)

# Visualize category performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Engagement rate by category
category_performance['engagement_rate'].plot(kind='barh', ax=axes[0], color='#E60023', alpha=0.7)
axes[0].set_title('Average Engagement Rate by Category', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Engagement Rate (%)')
axes[0].invert_yaxis()

# Total interactions by category
category_performance['total_interactions'].plot(kind='barh', ax=axes[1], color='#BD081C', alpha=0.7)
axes[1].set_title('Total Interactions by Category', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Total Interactions')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('../dashboards/category_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Content Type Performance

In [ ]:
# Analyze performance by content type
content_type_performance = pin_performance.groupby('content_type').agg({
    'pin_id': 'count',
    'engagement_rate': 'mean',
    'total_interactions': 'sum',
    'save': 'mean',
    'repin': 'mean',
    'click': 'mean'
}).rename(columns={'pin_id': 'total_pins'})

print("Content Type Performance:")
display(content_type_performance.sort_values('engagement_rate', ascending=False))

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Content Type Performance Analysis', fontsize=16, fontweight='bold')

# Engagement rate
content_type_performance['engagement_rate'].plot(kind='bar', ax=axes[0,0], color='coral', alpha=0.7)
axes[0,0].set_title('Avg Engagement Rate by Content Type')
axes[0,0].set_ylabel('Engagement Rate (%)')
axes[0,0].tick_params(axis='x', rotation=45)

# Average saves
content_type_performance['save'].plot(kind='bar', ax=axes[0,1], color='green', alpha=0.7)
axes[0,1].set_title('Avg Saves by Content Type')
axes[0,1].set_ylabel('Average Saves')
axes[0,1].tick_params(axis='x', rotation=45)

# Average repins
content_type_performance['repin'].plot(kind='bar', ax=axes[1,0], color='blue', alpha=0.7)
axes[1,0].set_title('Avg Repins by Content Type')
axes[1,0].set_ylabel('Average Repins')
axes[1,0].tick_params(axis='x', rotation=45)

# Average clicks
content_type_performance['click'].plot(kind='bar', ax=axes[1,1], color='purple', alpha=0.7)
axes[1,1].set_title('Avg Clicks by Content Type')
axes[1,1].set_ylabel('Average Clicks')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../dashboards/content_type_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. User Behavior Analysis and Segmentation

In [ ]:
# Calculate user-level metrics
user_behavior = interactions_df.groupby('user_id').agg({
    'interaction_id': 'count',
    'pin_id': 'nunique',
    'timestamp': ['min', 'max']
}).reset_index()

user_behavior.columns = ['user_id', 'total_interactions', 'unique_pins_interacted', 
                         'first_interaction', 'last_interaction']

# Calculate recency (days since last interaction)
max_date = pd.to_datetime(interactions_df['timestamp'].max())
user_behavior['recency_days'] = (max_date - pd.to_datetime(user_behavior['last_interaction'])).dt.days

# Calculate frequency (interactions per day)
user_behavior['activity_span_days'] = (
    pd.to_datetime(user_behavior['last_interaction']) - 
    pd.to_datetime(user_behavior['first_interaction'])
).dt.days + 1
user_behavior['frequency'] = user_behavior['total_interactions'] / user_behavior['activity_span_days']

# Count interaction types per user
user_interaction_types = interactions_df.groupby(['user_id', 'interaction_type']).size().unstack(fill_value=0)
user_behavior = user_behavior.merge(user_interaction_types, left_on='user_id', right_index=True, how='left')

# Merge with user demographics
user_behavior = user_behavior.merge(users_df[['user_id', 'user_type', 'age_group', 'country']], 
                                   on='user_id', how='left')

print("✓ User behavior metrics calculated!")
print(f"\nUser Behavior Summary:")
print(user_behavior[['total_interactions', 'unique_pins_interacted', 'recency_days', 'frequency']].describe())

## 7. RFM Analysis (Recency, Frequency, Monetary)

For Pinterest, we'll use engagement value instead of monetary value

In [ ]:
# Create RFM scores
# Recency: Lower is better (more recent)
# Frequency: Higher is better (more active)
# Engagement Value: saves + repins (higher value interactions)

user_behavior['engagement_value'] = (
    user_behavior.get('save', 0) + 
    user_behavior.get('repin', 0)
)

# Create quintile scores (1-5)
user_behavior['R_score'] = pd.qcut(user_behavior['recency_days'], 5, labels=[5,4,3,2,1], duplicates='drop')
user_behavior['F_score'] = pd.qcut(user_behavior['frequency'], 5, labels=[1,2,3,4,5], duplicates='drop')
user_behavior['M_score'] = pd.qcut(user_behavior['engagement_value'], 5, labels=[1,2,3,4,5], duplicates='drop')

# Convert to numeric
user_behavior['R_score'] = user_behavior['R_score'].astype(float)
user_behavior['F_score'] = user_behavior['F_score'].astype(float)
user_behavior['M_score'] = user_behavior['M_score'].astype(float)

# Calculate RFM score
user_behavior['RFM_score'] = (
    user_behavior['R_score'] + 
    user_behavior['F_score'] + 
    user_behavior['M_score']
) / 3

# Segment users
def rfm_segment(row):
    if row['RFM_score'] >= 4:
        return 'Champions'
    elif row['RFM_score'] >= 3:
        return 'Loyal Users'
    elif row['RFM_score'] >= 2.5:
        return 'Potential Loyalists'
    elif row['R_score'] >= 3 and row['RFM_score'] < 2.5:
        return 'Recent Users'
    elif row['F_score'] <= 2 and row['M_score'] <= 2:
        return 'At Risk'
    else:
        return 'Needs Attention'

user_behavior['segment'] = user_behavior.apply(rfm_segment, axis=1)

print("✓ RFM analysis complete!")
print(f"\nUser Segmentation:")
print(user_behavior['segment'].value_counts())
print(f"\nSegment Percentages:")
print(user_behavior['segment'].value_counts(normalize=True) * 100)

In [ ]:
# Visualize user segments
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('User Segmentation Analysis', fontsize=16, fontweight='bold')

# Segment distribution
segment_counts = user_behavior['segment'].value_counts()
axes[0,0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%')
axes[0,0].set_title('User Distribution by Segment')

# Average interactions by segment
segment_metrics = user_behavior.groupby('segment').agg({
    'total_interactions': 'mean',
    'engagement_value': 'mean',
    'frequency': 'mean'
})
segment_metrics['total_interactions'].plot(kind='barh', ax=axes[0,1], color='steelblue', alpha=0.7)
axes[0,1].set_title('Avg Interactions by Segment')
axes[0,1].set_xlabel('Average Interactions')

# Engagement value by segment
segment_metrics['engagement_value'].plot(kind='barh', ax=axes[1,0], color='green', alpha=0.7)
axes[1,0].set_title('Avg Engagement Value by Segment')
axes[1,0].set_xlabel('Average Engagement Value')

# Frequency by segment
segment_metrics['frequency'].plot(kind='barh', ax=axes[1,1], color='coral', alpha=0.7)
axes[1,1].set_title('Avg Daily Frequency by Segment')
axes[1,1].set_xlabel('Interactions per Day')

plt.tight_layout()
plt.savefig('../dashboards/user_segmentation.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. K-Means Clustering for Additional Insights

In [ ]:
# Prepare features for clustering
clustering_features = ['total_interactions', 'unique_pins_interacted', 'frequency', 
                       'engagement_value', 'recency_days']
X = user_behavior[clustering_features].fillna(0)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine optimal number of clusters using elbow method
inertias = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, marker='o', linewidth=2)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia', fontsize=12)
plt.title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.savefig('../dashboards/elbow_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Elbow curve plotted!")

In [ ]:
# Apply K-Means with optimal k (let's use k=4)
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
user_behavior['cluster'] = kmeans.fit_predict(X_scaled)

# Analyze clusters
cluster_analysis = user_behavior.groupby('cluster').agg({
    'user_id': 'count',
    'total_interactions': 'mean',
    'unique_pins_interacted': 'mean',
    'frequency': 'mean',
    'engagement_value': 'mean',
    'recency_days': 'mean'
}).rename(columns={'user_id': 'user_count'})

# Name clusters based on characteristics
cluster_names = {}
for idx, row in cluster_analysis.iterrows():
    if row['frequency'] > cluster_analysis['frequency'].median() and \
       row['engagement_value'] > cluster_analysis['engagement_value'].median():
        cluster_names[idx] = 'Power Users'
    elif row['total_interactions'] < cluster_analysis['total_interactions'].median() and \
         row['recency_days'] > cluster_analysis['recency_days'].median():
        cluster_names[idx] = 'Inactive Users'
    elif row['frequency'] > cluster_analysis['frequency'].median():
        cluster_names[idx] = 'Active Browsers'
    else:
        cluster_names[idx] = 'Casual Users'

user_behavior['cluster_name'] = user_behavior['cluster'].map(cluster_names)
cluster_analysis['cluster_name'] = cluster_analysis.index.map(cluster_names)

print("Cluster Analysis:")
display(cluster_analysis)

print("\nCluster Distribution:")
print(user_behavior['cluster_name'].value_counts())

## 9. Save Results and Key Insights

In [ ]:
# Save enhanced user behavior data
user_behavior.to_csv('../data/processed/user_behavior_segmented.csv', index=False)
print("✓ User behavior data saved!")

# Save pin performance data
pin_performance.to_csv('../data/processed/pin_performance.csv', index=False)
print("✓ Pin performance data saved!")

# Save category performance
category_performance.to_csv('../data/processed/category_performance.csv')
print("✓ Category performance data saved!")

## 10. Key Insights and Recommendations

In [ ]:
print("="*70)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("="*70)

print("\n1. CONTENT PERFORMANCE INSIGHTS:")
print("-" * 70)
top_category = category_performance.index[0]
top_engagement = category_performance.iloc[0]['engagement_rate']
print(f"   • Best performing category: {top_category} ({top_engagement:.2f}% engagement rate)")

best_content_type = content_type_performance['engagement_rate'].idxmax()
print(f"   • Best performing content type: {best_content_type}")

desc_perf = pin_performance.groupby('has_description')['engagement_rate'].mean()
print(f"   • Pins with descriptions have {desc_perf[True]/desc_perf[False]:.2f}x higher engagement")

print("\n2. USER SEGMENTATION INSIGHTS:")
print("-" * 70)
for segment in user_behavior['segment'].value_counts().index:
    count = len(user_behavior[user_behavior['segment'] == segment])
    pct = (count / len(user_behavior)) * 100
    avg_engagement = user_behavior[user_behavior['segment'] == segment]['engagement_value'].mean()
    print(f"   • {segment}: {count:,} users ({pct:.1f}%) - Avg Engagement: {avg_engagement:.1f}")

print("\n3. BEHAVIORAL PATTERNS:")
print("-" * 70)
peak_hour = interactions_df['hour'].mode()[0]
print(f"   • Peak engagement hour: {peak_hour}:00")

mobile_pct = (interactions_df['device'] == 'Mobile').sum() / len(interactions_df) * 100
print(f"   • Mobile usage: {mobile_pct:.1f}% of all interactions")

print("\n4. ACTIONABLE RECOMMENDATIONS:")
print("-" * 70)
print("   1. Prioritize {top_category} content in recommendations")
print("   2. Encourage creators to add descriptions (improves engagement)")
print("   3. Focus on {best_content_type} format for maximum engagement")
print("   4. Implement re-engagement campaigns for 'At Risk' users")
print("   5. Optimize mobile experience (70%+ of interactions)")
print("   6. Schedule high-priority content around peak hours")

print("\n" + "="*70)
print("ANALYSIS COMPLETE - Ready for Part 3: Predictive Modeling")
print("="*70)